#  Imports verses from Tanakh into a LaTeX-friendly format

In [1]:
#this defines which book is where and what the exact names are
#to look up the names properly

In [2]:
torah = 'torah.csv'
megillot = 'chamesh megillot.csv'
navi1 = 'neviim rishonim.csv'
navi2 = 'neviim achronim.csv'
emet = 'sifrei emet.csv'
ketuvim = 'ketuvim.csv'

In [3]:
sefarim = {
    "ספר בראשית":torah,
    "ספר שמות":torah,
    "ספר ויקרא":torah,
    "ספר במדבר":torah,
    "ספר דברים":torah,
    "ספר יהושע":navi1,
    "ספר שופטים":navi1,
    "ספר שמואל":navi1,
    "ספר מלכים":navi1,
    "ספר ישעיהו":navi2,
    "ספר ירמיהו":navi2,
    "ספר יחזקאל":navi2,
    "ספר תרי עשר":navi2,
    "ספר תהלים":emet,
    "ספר משלי":emet,
    "ספר איוב":emet,
    "מגילת שיר השירים":megillot,
    "מגילת רות":megillot,
    "מגילת איכה":megillot,
    "מגילת קהלת":megillot,
    "מגילת אסתר":megillot,
    "ספר דניאל":ketuvim
    #needs support for  aleph/bet, trei-asar, ezra-nehemiah, divrei-hayamim
}

In [4]:
sefer_name = {
    "gen":"ספר בראשית",
    "ex":"ספר שמות",
    "lev":"ספר ויקרא",
    "num":"ספר במדבר",
    "dev":"ספר דברים",
    "josh":"ספר יהושע",
    "judges":"ספר שופטים",
    "sam":"ספר שמואל",
    "kings":"ספר מלכים",
    "is":"ספר ישעיהו",
    "jer":"ספר ירמיהו",
    "ezek":"ספר יחזקאל",
    #"ספר תרי עשר",
    "psalms":"ספר תהלים",
    "prov":"ספר משלי",
    "job":"ספר איוב",
    "song":"מגילת שיר השירים",
    "ruth":"מגילת רות",
    "lam":"מגילת איכה",
    "ecc":"מגילת קהלת",
    "esther":"מגילת אסתר",
    "dan":"ספר דניאל"
    #needs support for  aleph/bet, trei-asar, ezra-nehemiah, divrei-hayamim
}

In [5]:
#this parses the formatting notes in the 3rd column of the spreadsheet

In [6]:
format = {
    "//{{פפ}}//":"\p",
    "{{סס}}":"\s",
    "{{פפ}}":"\p",
    "{{פפפ}}":"\p",
    "{{רווח בסוף שורה}}":"\p",
    "{{ססס}}":"\s",
    "{{סס2}}":"\s",
    "//{{סס}}":"\s",
    "{{פפ}}//":"\p",
    "{{מ:אין פרשה בתחילת פרק}}":"",
    "{{ר4}}":"\hfill"
    #{{פסקא באמצע פסוק|משתנה|הערות}}
}

In [7]:
def format_parser(format_in):
    blank = '__'
    new_book = 'ספר חדש'
    
    if format_in == blank:
        return ''
    elif format_in in format:
        return format[format_in]
    elif new_book in format_in:
        index = format_in.find('|')
        book_name_start = index+1
        book_name_end = format_in.find('}',index)
        book_name = format_in[book_name_start:book_name_end]
        return '\\newbook{'+book_name+'}'
    else:
        return format_in

In [8]:
#this looks for a note about the aliya in the 4th column of the spreadsheet.
#maybe functionality for sedarim will be added over time.

In [9]:
#this would change aliyot info to exact commands
#the current plan is to just have the aliya text be the argument of the \aliya{} command, so this is not needed.
#if used, this needs support for double parshiyot.
aliya_commands = {
    "[לוי]":"\\aliya{levi}",
    "[ישראל]":"\\aliya{yisrael}",
    "[ע\"כ ישראל]":"\\aliya{end_yisrael}",
    "שני":"\\aliya{sheni}",
    "שלישי":"\\aliya{shelishi}",
    "רביעי":"\\aliya{revii}",
    "חמישי":"\\aliya{chamishi}",
    "ששי":"\\aliya{shishi}",
    "שביעי":"\\aliya{shevii}",
    "מפטיר":"\\aliya{maftir}"
}

In [10]:
def aliya_parser(aliya_in):
    aliya = 'עלייה'
    aliya_command = ''
    seder_command = ''
    output = ''
    if aliya in aliya_in:
        aliya_start=aliya_in.find(aliya)
        while aliya_start<len(aliya_in) and aliya_in[aliya_start]!='=':
            aliya_start=aliya_start+1
        aliya_end = aliya_start+1
        while aliya_end<len(aliya_in) and aliya_in[aliya_end]!='}':
            aliya_end = aliya_end+1
        aliya_number = aliya_in[aliya_start+1:aliya_end]
        if '\'' in aliya_number:
            parsha_name = aliya_number.replace('\'','')
            aliya_command = '\\parsha{'+parsha_name+'}'
        else:
            aliya_command = '\\aliya{'+aliya_number+'}'
    
    seder = 'סדר'
    if seder in aliya_in:
        seder_start=aliya_in.find(seder)
        while seder_start<len(aliya_in) and aliya_in[seder_start]!='=':
            seder_start=seder_start+1
        seder_end = seder_start+1
        while seder_end<len(aliya_in) and aliya_in[seder_end]!='|':
            seder_end = seder_end+1
        seder_command = '\seder{'+aliya_in[seder_start+1:seder_end]+'}'
    
    output = seder_command + aliya_command
    return output

In [11]:
#this parses notes embedded in the tanakh-text.

In [12]:
text_format = {
    "{{מ:לגרמיה}}":" ׀ ",
    "{{מ:פסק}}":" ׀ "
    
}

In [13]:
def nusach_parser(note):
    main_nusach_start = note.find('|')+1
    main_nusach_end = note.find('|',main_nusach_start+1)
    main_nusach = note[main_nusach_start:main_nusach_end]
    
    if '=' in note:
        secondary_nusach_start = note.find('=')+1
    else:
        secondary_nusach_start = note.find('|',main_nusach_end-1)+1
        
    secondary_nusach = note[secondary_nusach_start:len(note)-2]
    #output = main_nusach+'\\footnote{'+secondary_nusach+'}'
    output = main_nusach
    return output

In [14]:
def text_parser(pasuk):
    #this loop replaces the items which can be replaced by a specific character(s) using the text_format dict
    for entry in text_format:
        if entry in pasuk:
            pasuk = pasuk.replace(entry,text_format[entry])
    
    #this parses the nusach-notes and turns them into footnotes, using the nusach_parser
    nusach = 'נוסח'
    while nusach in pasuk:
        index = pasuk.find(nusach)
        note_start = index-2
        extra_brace = 0
        while index<len(pasuk) and pasuk[index]!='}':
            if pasuk[index]=='{':
                extra_brace = extra_brace+1
            index = index+1
        while index<len(pasuk) and extra_brace != 0:    
            if pasuk[index]=='}':
                extra_brace = extra_brace-1
            index = index+1
        note_end = pasuk.find('}}',index)+2
        nusach_note = pasuk[note_start:note_end]
        nusach_new = nusach_parser(nusach_note)
        pasuk = pasuk.replace(nusach_note,nusach_new)
    
    yerush = '{{מ:ירושלם'
    while yerush in pasuk:
        index = pasuk.find(yerush)
        vowel_1_loc = pasuk.find('ם',index)+2
        vowel_1 = pasuk[vowel_1_loc]
        ZWJ = '‍'
        trop_loc = vowel_1_loc + 2
        trop = pasuk[trop_loc]
        if trop=='|':
            trop = ''
        correct_text = 'ל'+vowel_1+trop+ZWJ+'ִ'
        yerush_end = pasuk.find('ם',index+10)
        yerush_old = pasuk[index-1:yerush_end]
        pasuk = pasuk.replace(yerush_old,correct_text)

    kq = 'כו"ק'
    qk = 'קו"כ'
    qk_em='קו"כ-אם'
    klq = 'כתיב ולא קרי'
    qlk = 'קרי ולא כתיב'
    if (kq or qk) in pasuk:
        if kq in pasuk:
            index = pasuk.find(kq)
        elif qk in pasuk:
            index = pasuk.find(qk)
        kq_start = index-2
        kq_end = pasuk.find('}}',index+3)+2
        kq_note = pasuk[kq_start:kq_end]
        arg1_start = pasuk.find('|',index)+1
        arg1_end = pasuk.find('|',arg1_start+1)
        arg2_start = arg1_end+2
        arg2_end = kq_end-2
        arg1 = pasuk[arg1_start:arg1_end]
        arg2 = pasuk[arg2_start:arg2_end]
        if kq in pasuk:
            k = arg1
            q = arg2
        elif qk in pasuk:
            q = arg1
            k = arg2
        #put something to include ketivs for Tanakh
        #this is for the siddur, which uses only qere
        pasuk = pasuk.replace(kq_note,q)
    while qk_em in pasuk:
        break
    while klq in pasuk:
        break
    while qlk in pasuk:
        break
        
    kamatz_note = 'מ:קמץ'
    if kamatz_note in pasuk:
        index = pasuk.find(kamatz_note)
        note_start = index-2
        note_end = pasuk.find('}}',index)+2
        note_to_remove = pasuk[note_start:note_end]
        text_end = note_to_remove.find('|',10)
        final_text = note_to_remove[10:text_end]
        pasuk = pasuk.replace(note_to_remove,final_text)
    
    #needs special characters
    #needs words with alternate kamatz katan forms
    return pasuk

In [15]:
#this pulls verses from the Torah and runs them through the above parsers

In [16]:
import csv

book_in = 'prov'
book_in = sefer_name[book_in]
chelek = sefarim[book_in]

perek_start = 'לא'
perek_end = 'לא'
pasuk_start = 'א'
pasuk_end = 'ל'
output = ''
output_file = 'output.txt'

with open(chelek, 'r', encoding='utf-8') as csvDataFile, open(output_file,'w',encoding='utf-8') as outfile:
    csvReader = csv.reader(csvDataFile)
    in_range = False
    for row in csvReader:
        if row[1]=='0' or row[1]=='תתת':
            continue
        book_perek = row[0].split('/')
        pasuk = row[1]
        if book_in == book_perek[0] and perek_start==book_perek[1] and pasuk_start==pasuk:
            in_range = True
        
        if in_range==True:
            if pasuk == 'א':
                perek_command = '\\perek{'+book_perek[1]+'}'
            else:
                perek_command = ''
            par = format_parser(row[2])+perek_command+'\n'
            aliya = aliya_parser(row[3])+'\n'
            text = text_parser(row[4])
            line = par+aliya+text
            print(line)
            outfile.write(line)        
        if perek_end==book_perek[1] and pasuk_end==pasuk:
            break

\p\perek{לא}

דִּ֭בְרֵי לְמוּאֵ֣ל מֶ֑לֶךְ{{ר1}}מַ֝שָּׂ֗א אֲֽשֶׁר־יִסְּרַ֥תּוּ אִמּֽוֹ׃
\hfill

מַה־בְּ֭רִי וּמַֽה־בַּר־בִּטְנִ֑י{{ר1}}וּ֝מֶ֗ה בַּר־נְדָרָֽי׃
\hfill

אַל־תִּתֵּ֣ן לַנָּשִׁ֣ים חֵילֶ֑ךָ{{ר1}}וּ֝דְרָכֶ֗יךָ לַֽמְח֥וֹת מְלָכִֽין׃
\hfill

אַ֤ל לַֽמְלָכִ֨ים ׀ לְֽמוֹאֵ֗ל{{ר1}}אַ֣ל לַֽמְלָכִ֣ים שְׁתוֹ־יָ֑יִן{{ר2}}וּ֝לְרוֹזְנִ֗ים ֵ֣י שֵׁכָֽר׃
\hfill

פֶּן־יִ֭שְׁתֶּה וְיִשְׁכַּ֣ח מְחֻקָּ֑ק{{ר1}}וִ֝ישַׁנֶּ֗ה דִּ֣ין כׇּל־בְּנֵי־עֹֽנִי׃
\hfill

תְּנוּ־שֵׁכָ֣ר לְאוֹבֵ֑ד{{ר1}}וְ֝יַ֗יִן לְמָ֣רֵי נָֽפֶשׁ׃
\hfill

יִ֭שְׁתֶּה וְיִשְׁכַּ֣ח רִישׁ֑וֹ{{ר1}}וַ֝עֲמָל֗וֹ לֹ֣א יִזְכׇּר־עֽוֹד׃
\p

פְּתַח־פִּ֥יךָ לְאִלֵּ֑ם{{ר1}}אֶל־דִּ֗֝ין כׇּל־בְּנֵ֥י חֲלֽוֹף׃
\hfill

פְּתַח־פִּ֥יךָ שְׁפׇט־צֶ֑דֶק{{ר1}}וְ֝דִ֗ין עָנִ֥י וְאֶבְיֽוֹן׃
\p

אֵֽשֶׁת־חַ֭יִל מִ֣י יִמְצָ֑א{{ר1}}וְרָחֹ֖ק מִפְּנִינִ֣ים מִכְרָֽהּ׃
\hfill

בָּ֣טַח בָּ֭הּ לֵ֣ב בַּעְלָ֑הּ{{ר1}}וְ֝שָׁלָ֗ל לֹ֣א יֶחְסָֽר׃
\hfill

גְּמָלַ֣תְהוּ ט֣וֹב וְלֹא־רָ֑ע{{ר1}}כֹּ֗֝ל יְמֵ֣י חַיֶּֽיהָ׃
\hfill

דָּ֭רְשָׁה צֶ֣מֶר וּפִשְׁתִּ֑ים{{ר1}}וַ֝תַּ֗עַשׂ בְּחֵ֣

In [17]:
def psalm_parser(pasuk):
    pasuk = pasuk.replace('{{ר1}}',' ')
    pasuk = pasuk.replace('{{ר0}}',' ')   
    pasuk = pasuk.replace('{{ר2}}',' ')
    pasuk = pasuk.replace('{{ר3}}',' ')
    pasuk = pasuk.replace('{{אתנח הפוך}}','֢')
    pasuk = pasuk.replace('//',' ')
    return pasuk

In [28]:
import csv

book_in = 'psalms'
book_in = sefer_name[book_in]
chelek = sefarim[book_in]

perek = 'צב'
output = ''
output_file = 'output.txt'
print('\\firstword{}\source{תהלים'," ",perek,'}%',sep="")
psalm_output = ''

with open(chelek, 'r', encoding='utf-8') as csvDataFile, open(output_file,'w',encoding='utf-8') as outfile:
    csvReader = csv.reader(csvDataFile)
    in_range = False
    for row in csvReader:
        if row[1]=='0' or row[1]=='תתת':
            continue
        book_perek = row[0].split('/')
        pasuk = row[1]
        if book_in == book_perek[0] and perek==book_perek[1]:
            in_range = True
        else:
            in_range = False

        if in_range==True:
            text = text_parser(row[4])
            text = psalm_parser(text)
            print(text)
            psalm_output = psalm_output + text

\firstword{}\source{תהלים צב}%
מִזְמ֥וֹר שִׁ֗יר לְי֣וֹם הַשַּׁבָּֽת׃
ט֗וֹב לְהֹד֥וֹת לַיהֹוָ֑ה וּלְזַמֵּ֖ר לְשִׁמְךָ֣ עֶלְיֽוֹן׃
לְהַגִּ֣יד בַּבֹּ֣קֶר חַסְדֶּ֑ךָ וֶ֝אֱמ֥וּנָתְךָ֗ בַּלֵּילֽוֹת׃
עֲֽלֵי־עָ֭שׂוֹר וַעֲלֵי־נָ֑בֶל עֲלֵ֖י הִגָּי֣וֹן בְּכִנּֽוֹר׃
כִּ֤י שִׂמַּחְתַּ֣נִי יְהֹוָ֣ה בְּפׇעֳלֶ֑ךָ בְּֽמַעֲשֵׂ֖י יָדֶ֣יךָ אֲרַנֵּֽן׃
מַה־גָּדְל֣וּ מַעֲשֶׂ֣יךָ יְהֹוָ֑ה מְ֝אֹ֗ד עָמְק֥וּ מַחְשְׁבֹתֶֽיךָ׃
אִֽישׁ־בַּ֭עַר לֹ֣א יֵדָ֑ע וּ֝כְסִ֗יל לֹא־יָבִ֥ין אֶת־זֹֽאת׃
בִּפְרֹ֤חַ רְשָׁעִ֨ים ׀ {{ר0|רווח=לא}}כְּמ֥וֹ עֵ֗שֶׂב וַ֭יָּצִיצוּ כׇּל־פֹּ֣עֲלֵי אָ֑וֶן לְהִשָּׁמְדָ֥ם עֲדֵי־עַֽד׃
וְאַתָּ֥ה מָר֗וֹם לְעֹלָ֥ם יְהֹוָֽה׃
כִּ֤י הִנֵּ{{גלגל}}ה אֹיְבֶ֡יךָ ׀ יְֽהֹוָ֗ה כִּֽי־הִנֵּ֣ה אֹיְבֶ֣יךָ יֹאבֵ֑דוּ יִ֝תְפָּרְד֗וּ כׇּל־פֹּ֥עֲלֵי אָֽוֶן׃
וַתָּ֣רֶם כִּרְאֵ֣ים קַרְנִ֑י בַּ֝לֹּתִ֗י בְּשֶׁ֣מֶן רַעֲנָֽן׃
וַתַּבֵּ֥ט עֵינִ֗י בְּשׁ֫וּרָ֥י בַּקָּמִ֖ים עָלַ֥י מְרֵעִ֗ים תִּשְׁמַ֥עְנָה אׇזְנָֽי׃
צַ֭דִּיק כַּתָּמָ֣ר יִפְרָ֑ח כְּאֶ֖רֶז בַּלְּבָנ֣וֹן יִשְׂגֶּֽה׃
שְׁ֭תוּלִים בְּבֵ֣ית יְהֹוָ֑ה בְּחַצְר֖וֹת אֱלֹהֵ֣ינו